In [9]:
import gzip, json
from tqdm import tqdm

In [10]:
def shrink_wiktextract_entry(entry: dict) -> dict | None:
    """
    Keep only full keys relevant to etymological relationships.
    If the entry has no etymology/derivational info at all, return None.
    """

    # Main etymology-bearing keys
    target_keys = {
        "word",
        "lang",
        "lang_code",
        "pos",
        "etymology_number",
        "etymology_text",
        "etymology_templates",
        "derived",
        "descendants",
        "alt_of",
        "form_of",
        "related",
        "categories",
    }

    # Filter condition: does this entry have anything we care about?
    has_info = any(
        key in entry and entry[key]
        for key in ("etymology_text", "etymology_templates", "derived", "descendants")
    )

    if not has_info:
        return None  # discard entry entirely

    # Build output using full unmodified values
    out = {}
    for key in target_keys:
        if key in entry:
            out[key] = entry[key]  # keep full content of the key

    return out


In [11]:
in_path  = "/content/raw-wiktextract-data.jsonl.gz"
out_path = "/content/filtered-etymology.jsonl.gz"

count_keep = 0
count_skip = 0
lines_processed = 0

with gzip.open(in_path, "rt", encoding="utf-8") as fin, \
     gzip.open(out_path, "wt", encoding="utf-8") as fout:

    try:
        for line in tqdm(fin):
            lines_processed += 1
            entry = json.loads(line)

            slim = shrink_wiktextract_entry(entry)

            if slim is None:
                count_skip += 1
            else:
                count_keep += 1
                fout.write(json.dumps(slim, ensure_ascii=False,
                                      separators=(",", ":")) + "\n")

    except EOFError:
        print(f"\nWARNING: truncated gzip: unexpected EOF after {lines_processed} lines.")

2569612it [05:15, 8132.66it/s] 

In [12]:
print("keep:", count_keep)
print("skip:", count_skip)
print("lines processed:", lines_processed)
print("written to:", out_path)

keep: 860080
skip: 1709532
lines processed: 2569612
written to: /content/filtered-etymology.jsonl.gz
